# Example Create Agent using Gen AI SDK

**Authors**: 
- Muhammad Afif Al hawari (muhammad.a.a.hawari@gdplabs.id)

**Reviewers**: 
- Villia Susanti (villia.susanti@gdplabs.id)

## Summary
In this Notebook we will try to create a sample agent using GenAI SDK and Open AI GPT 3.5. The Agent will use two tools, one is Search Tool which utilize google search API and multiplier tool which takes two numbers and multipliy them.

## References
- [Langchain Agent Docs](https://python.langchain.com/docs/modules/agents/)
- [Gen AI Agent Orchestrator](https://docs.glair.ai/generative-internal/modules/agent-orchestrator)

## Prepare Environment

### Install Dependencies

In [ ]:
pip install google-search-results

### Install Gen AI SDK

To install the SDK library, you need to create a personal access token on GitHub. Please follow these steps:
1. You need to log in to your [GitHub Account](https://github.com/).
2. Go to the [Personal Access Tokens](https://github.com/settings/tokens) page.
3. If you haven't created a Personal Access Tokens yet, you can generate one.
4. When generating a new token, make sure that you have checked the `repo` option to grant access to private repositories.
5. Now, you can copy the new token that you have generated and paste it into the script below.

In [ ]:
import getpass
import subprocess
import sys
import os

def install_sdk_library():
    token = getpass.getpass("Input Your Personal Access Token: ")

    cmd = f"pip install -e git+https://{token}@github.com:GDP-ADMIN/gen-ai-internal.git#egg=gdplabs_gen_ai"

    with subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as process:
        stdout, stderr = process.communicate()

        if process.returncode != 0:
            sys.stdout.write(stderr)
            raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
        else:
            sys.stdout.write(stdout)

install_sdk_library()

### Add Environment Variable

You will need two provide two API Keys. OpenAI API Key and SerpAPI API Key.

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI API KEY: ")
os.environ["SERPAPI_API_KEY"] = getpass.getpass("SERPERAI API KEY: ")

## Create and execute Agent

### Define Tools for the Agent

In this section, we will create two tools that will be used for our agent determining the answer for the query/question given by user. 
1. `Search Tool` - Utilize Google search API to get additional information
2. `Multiplier Tool` - Accept two numbers and return the result of their multiplication

To define the tools, we use interface on Gen AI SDK. You can read more details on [Agent Tools](https://docs.glair.ai/generative-internal/modules/agent-orchestrator/tools).

In [ ]:
from gdplabs_gen_ai.agent_orchestrator.agent_executor import initialize_agent_executor
from gdplabs_gen_ai.agent_orchestrator.agent_type import AgentType
from gdplabs_gen_ai.agent_orchestrator.tools import BaseTool
from gdplabs_gen_ai.inference_orchestrator.llm import OpenAILLM
from langchain.utilities import SerpAPIWrapper

def parsing_multiplier(string):
    a, b = string.split(",")
    return multiplier(int(a), int(b))

def multiplier(a, b):
    return a * b

search = SerpAPIWrapper()

multiplier_tool = BaseTool(
    name="Multiplier Search",
    func=parsing_multiplier,
    description="useful for when you need to multiply two numbers together. The input to this tool should be a comma separated list of numbers of length two, representing the two numbers you want to multiply together. For example, `1,2` would be the input if you wanted to multiply 1 by 2."
)

search_tool = BaseTool(
    func=search.run,
    name="Search",
    description="useful for when you need to answer questions about current events",
)

### Create Agent Executor

In [ ]:
openai_llm = OpenAILLM("gpt-3.5-turbo")

agent_executor = initialize_agent_executor(
    tools=[multiplier_tool, search_tool],
    llm=openai_llm.llm,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

### Execute Query on the Agent

In [7]:
agent_executor.run_query(query="Who is Leo DiCaprio's girlfriend? What is her current age multiplied by 3?")



> Entering new AgentExecutor chain...
I need to find out Leo DiCaprio's girlfriend's name first, and then multiply her current age by 3.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Vittoria Ceretti
Thought:Now that I know Leo DiCaprio's girlfriend's name, I can find out her current age.
Action: Search
Action Input: "Vittoria Ceretti age"
Observation: 25 years
Thought:Now that I know Vittoria Ceretti's current age, I can multiply it by 3.
Action: Multiplier Search
Action Input: "25,3"
Observation: 75
Thought:I now know the final answer
Final Answer: Vittoria Ceretti's current age multiplied by 3 is 75.

> Finished chain.


"Vittoria Ceretti's current age multiplied by 3 is 75."